In [8]:
"""

Check how the model pads input - done
Check how different enecoding & decoding params affect the encoding & decoding - done
check performance of model on a sample set at different batch sizes (inadvertently check how the model performs when padding is present) - till now 1 bs gives best performance

"""

# Autocompletion
%config Completer.use_jedi = False

# Autoreload
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('/workspace/llmsearch')

import gc
import torch
import ctypes
import json
import math
import torch
import random
import evaluate
import datasets
import numpy as np
import pandas as pd
import transformers
from transformers import GPTQConfig, BitsAndBytesConfig
from transformers.modeling_outputs import CausalLMOutputWithPast
from transformers import PreTrainedModel, PretrainedConfig, GenerationConfig, StoppingCriteria, AutoTokenizer, StoppingCriteriaList, AutoModel, AutoModelForCausalLM

import os
import gc
import ctypes
import traceback
from pathlib import Path
from typing import Any, Dict, Optional, Union, List

import time
import textwrap
from tqdm.auto import tqdm

from datasets import load_dataset
from llmsearch.utils.model_downloader import download_model_from_hf
from llmsearch.utils.model_utils import batcher, decoder_parser

import awq

from awq import AutoAWQForCausalLM

def pretty_print_dict(d, indent = 4):
    print(json.dumps(d, indent = indent, default = str))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
gsm8k_dataset = load_dataset("gsm8k", 'main')

torch.__version__, awq.__version__

('2.2.0+cu121', '0.2.4')

In [10]:

def seed_everything(seed):
    """Seed for reproducibilty"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False


def cm():
    gc.collect()
    ctypes.CDLL("libc.so.6").malloc_trim(0)
    torch.cuda.empty_cache()

def seed_everything(seed):
    """Seed for reproducibilty"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False



def perform_single_example_inference(example, model, tokenizer,gen_kwargs):

    tokenized_input = tokenizer(example, return_tensors = "pt", add_special_tokens = False)
    tokenized_input['input_ids'] = tokenized_input['input_ids'].to('cuda:0')

    tokenized_input['attention_mask'] = tokenized_input['attention_mask'].to('cuda:0')
    # tokenized_input.to(device)
    # print(tokenized_input)

    model_out = model.generate(**tokenized_input, **gen_kwargs)
    prompt_tokens = len(tokenized_input['input_ids'][0])
    print(f"Prompt tokens - {prompt_tokens}")
    # print(model_out.tolist()[0])

    output_token_ids = model_out.tolist()[0]
    decoded_output = tokenizer.decode(output_token_ids, spaces_between_special_tokens = False)

    print(decoded_output)
    completion_tokens = len(output_token_ids) - prompt_tokens

    print(f"Completion Tokens - {completion_tokens}")

    return decoded_output, prompt_tokens, completion_tokens

In [11]:
# loaders
def load_model_with_awq_backend(model_id, model_loader_kwargs, tokenizer_kwargs,temp_model_dir, model_branch = "main"):
    output_folder = download_model_from_hf(model_id, save_dir = temp_model_dir, branch = model_branch)

    model_loader_kwargs['pretrained_model_name_or_path'] = output_folder
    tokenizer_loader_kwargs['pretrained_model_name_or_path'] = output_folder

    model_name_or_path = model_loader_kwargs.pop('pretrained_model_name_or_path')
    model = AutoAWQForCausalLM.from_quantized(
        quant_path=model_name_or_path,
        **model_loader_kwargs
    )
    tokenizer = AutoTokenizer.from_pretrained(**tokenizer_kwargs, local_files_only=True)

    return model, tokenizer

model_loader_backend_map = {
    # "exllama_2_hf": load_model_with_exllama_2_hf_backend,
    # "hf": load_model_with_hf_backend,
    # 'auto_gptq' : load_model_with_autogptq_backend,
    'awq' : load_model_with_awq_backend,
}

In [12]:
# https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-AWQ
model_id = "TheBloke/CapybaraHermes-2.5-Mistral-7B-AWQ"

temp_model_dir = Path(f"/workspace/temp_model_dir/")
temp_model_dir.mkdir(exist_ok = True, parents = True)

model_loader_kwargs = {
    'device_map' : {'' : 0},
    'fuse_layers' : True,
}

tokenizer_loader_kwargs = {
    'use_fast' : False,
    'legacy' : False,
}

model, tokenizer = load_model_with_awq_backend(model_id, model_loader_kwargs, tokenizer_loader_kwargs,temp_model_dir, model_branch = "main")

# pad token is null in config -https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-AWQ/blob/eb64c310c44905321d012962db9ac0d47c3a64fa/tokenizer_config.json#L53



tokenizer.pad_token = tokenizer.unk_token
tokenizer.padding_side = 'left'

100%|██████████| 17.9k /17.9k  27.4MiB/s
100%|██████████| 911   /911    3.49MiB/s
100%|██████████| 51.0  /51.0   178kiB/s
100%|██████████| 126   /126    641kiB/s
100%|██████████| 420   /420    975kiB/ss
100%|██████████| 115   /115    493kiB/s
  0%|          | 15.7M /4.15G  46.3MiB/s
100%|██████████| 493k  /493k   41.5MiB/s

100%|██████████| 1.60k /1.60k  4.87MiB/s
100%|██████████| 1.80M /1.80M  3.32MiB/s
100%|██████████| 4.15G /4.15G  235MiB/s
Fusing layers...: 100%|██████████| 32/32 [00:02<00:00, 11.27it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
def preprocess_dataset(dataset, tokenizer, pt, pt_cols, system_prompt, add_generation_prompt = True):

    def wrapper(sample):
        """Takes in a sample, formats it using prompt template, applies chat template and returns the formatted string"""
        messages = [] if system_prompt is None else [{"role": "system", "content": system_prompt}]
        formatted_pt = pt.format(**{pt_col : sample[pt_col] for pt_col in pt_cols})
        messages.append(
            {
                "role": "user",
                "content": formatted_pt,
            }
        )
        formatted_pt_with_ct = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt=add_generation_prompt)
        return formatted_pt_with_ct

    def actual_input(sample):
        """Takes in a sample, formats it using prompt template, applies chat template and returns the formatted string"""
        return sample[pt_cols[0]]



    pt_dataset = dataset.map(
        lambda sample : {
            "X" : wrapper(sample),
            'actual input' : actual_input(sample),
        }
    )

    return pt_dataset

In [14]:
pt = textwrap.dedent("""\
    Q: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?
    A: There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6. The answer is 6.

    Q: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
    A: There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5. The answer is 5.

    Q: {question}""")
pt_cols = ['question']
system_prompt = "Solve the following math problems, end with The answer is"

# Add prompt template
processed_dataset = preprocess_dataset(gsm8k_dataset['train'], tokenizer,pt = pt, pt_cols = pt_cols, system_prompt = system_prompt, add_generation_prompt = True)

Map: 100%|██████████| 7473/7473 [00:00<00:00, 10970.22 examples/s]


In [15]:
import re


def get_score(y_true, y_pred):
    def extract_answer_from_out(s):
        pattern = re.compile(r"The answer is (\d+(?:\.\d+)?)")
        match = pattern.search(s)
        if match:
            return match.group(1).strip()
        else:
            return None

    scores = []

    for y_t, y_p in zip(y_true, y_pred):
        y_t_answer = y_t["answer"].split("####")[-1].strip()
        y_p_answer = extract_answer_from_out(y_p)

        # print("y_pred - ", y_p_answer)
        # print("y_true - ", y_t_answer)

        if y_t_answer == y_p_answer:
            scores.append(1)
        else:
            scores.append(0)
    return sum(scores) / len(scores)

In [16]:
from llmsearch.utils.logging_utils import set_verbosity_info, set_verbosity_debug, set_verbosity_warning
set_verbosity_info()

In [17]:
seed = 42
bm_sample_size = 50
bm_samples = processed_dataset.shuffle(seed = seed).select(range(bm_sample_size))

In [19]:
from llmsearch.scripts.stopping_criteria import MultiTokenStoppingCriteria

In [22]:
multi_token_stop_criteria_ob = MultiTokenStoppingCriteria(sequence_ids=[32000])

In [21]:

from llmsearch.tuner import Tuner

batch_size_list = [1, 2, 4, 8, 16, 32]
score_at_diff_batch_sizes = {}

for batch_size in batch_size_list:
    print(f"Batch Size - {batch_size}")

    cm()

    stopping_criteria = StoppingCriteriaList([multi_token_stop_criteria_ob])

    callbacks_after_inference = [multi_token_stop_criteria_ob.reset]

    tuner_ob = Tuner(
        model=model,
        tokenizer=tokenizer,
        dataset=bm_samples,
        device="cuda:0",
        batch_size=batch_size,
        tokenizer_encode_args={"padding": "longest", "add_special_tokens": False},
        tokenizer_decode_args={"spaces_between_special_tokens": False},
        scorer=get_score,
        prompt_template="{X}",
        seed=seed,
        column_mapping={"input_cols": ["X"], "eval_cols": ["answer"]},
        callbacks_after_inference=callbacks_after_inference,
    )


    # is stopping criteria cache not being reset properly?
    # check if reset condition is working properly


    gen_params1 = {
        'max_new_tokens' : 500,
        # max_new_tokens take precendece over stopping criteria
        'stopping_criteria' : stopping_criteria,
        'generation_seed' : 42,
    }

    start = time.time()
    scores, outputs = tuner_ob.get_score(gen_params1)
    end = time.time()

    score_at_diff_batch_sizes[batch_size] = {
        'score' : scores,
        'outputs' : outputs,
        'optimal_batch_size' : tuner_ob.estimator._optimal_batch_size,
        'latency_mins' : (end  - start) / 60,
    }

    print(f'score at batch size - {batch_size} - {scores}')

# Done - Problem - Stopping criteria does not reset when generation reaches max new tokens so no stop token is generated
# When does generation stop - max new tokens is reached or stop token is generated

# 2m44s for 50 samples
# 100 samples - 5m28s

Batch Size - 1


NameError: name 'multi_token_stop_criteria_ob' is not defined

In [ ]:
tokenizer.padding_side, tokenizer.pad_token

In [ ]:
for batch_size in score_at_diff_batch_sizes.keys():
    print(f"Batch Size - {batch_size}")
    print(f"Score - {score_at_diff_batch_sizes[batch_size]['score']}")
    print('\n\n')

In [ ]:
score_at_diff_batch_sizes[4]['outputs']

In [ ]:
# pad token = eos token
# padding side - left

for batch_size in score_at_diff_batch_sizes.keys():
    print(f"Batch Size - {batch_size}")
    print(f"Score - {score_at_diff_batch_sizes[batch_size]['score']}")
    print('\n\n')

In [ ]:
len(tokenizer(bm_samples[0]['X'], **{'padding': 'longest', 'add_special_tokens' : False})['input_ids'])

In [ ]:
score_at_diff_batch_sizes[4]['outputs']

In [ ]:
# print out the results

# proves that llmsearch should be run on a batch size of 1

for batch_size, score_dict in score_at_diff_batch_sizes.items():
    print(f"Batch Size - {batch_size}")
    print(f"Score - {score_dict['score']}")
    print(f"Latency - {score_dict['latency_mins']} mins")
    print(f"Optimal Batch Size - {score_dict['optimal_batch_size']}")
    print("\n\n")

In [ ]:
score_at_diff_batch_sizes

In [ ]:
score_at_diff_batch_sizes[batch_size] = {
        'score' : scores,
        'outputs' : outputs,
        'optimal_batch_size' : tuner_ob.estimator._optimal_batch_size,
        'latency_mins' : (end  - start) / 60,
    }

In [ ]:






print(scores_before)

In [ ]:
from llmsearch.tuner import Tuner

cm()
batch_size = 1

tuner_ob = Tuner(
    model = model,
    tokenizer = tokenizer,
    dataset = bm_samples,
    device = 'cuda:0',
    batch_size = batch_size,
    tokenizer_encode_args={'padding': 'longest', 'add_special_tokens' : False},
    tokenizer_decode_args={'spaces_between_special_tokens' : False},
    scorer = get_score,
    prompt_template = langchain.PromptTemplate.from_template("{X}"),
    is_encoder_decoder = False,
    seed = seed,
    column_mapping = {'input_cols' : ["X"],'eval_cols' : ['answer']},
)

In [ ]:
tuner_ob._optimal_batch_size

In [ ]:






print(scores_before)

In [ ]:
scores_before

In [ ]:
# decoder parser is working as expected
# TODO : check scores at different bs then llmsearch

stopping_criteria = StoppingCriteriaList([MultiTokenEOSCriteria(sequence_ids = [32000])])

gen_params1 = {
    'max_new_tokens' : 500,
    'stopping_criteria' : stopping_criteria,
    'generation_seed' : 42,
}

scores_before, outputs_before = tuner_ob.get_score(gen_params1)

scores_before